The Nobel Prize has been among the most prestigious international awards since 1901. Each year, awards are bestowed in chemistry, literature, physics, physiology or medicine, economics, and peace. In addition to the honor, prestige, and substantial prize money, the recipient also gets a gold medal with an image of Alfred Nobel (1833 - 1896), who established the prize.

![](Nobel_Prize.png)

The Nobel Foundation has made a dataset available of all prize winners from the outset of the awards from 1901 to 2023. The dataset used in this project is from the Nobel Prize API and is available in the `nobel.csv` file in the `data` folder.

In this project, you'll get a chance to explore and answer several questions related to this prizewinning data. And we encourage you then to explore further questions that you're interested in!

In [2]:
# Loading in required libraries
import pandas as pd
import seaborn as sns
import numpy as np

# Start coding here!

In [3]:
df = pd.read_csv('./data/nobel.csv')

top_gender = df['sex'].mode().values[0]
top_country = df['birth_country'].mode().values[0]

df.columns

Index(['year', 'category', 'prize', 'motivation', 'prize_share', 'laureate_id',
       'laureate_type', 'full_name', 'birth_date', 'birth_city',
       'birth_country', 'sex', 'organization_name', 'organization_city',
       'organization_country', 'death_date', 'death_city', 'death_country'],
      dtype='object')

In [4]:
df['decade'] = (df['year'] // 10) * 10
total_award_by_decade = df.groupby('decade')['prize'].count().reset_index()
total_award_by_decade = total_award_by_decade.rename(columns={'prize': 'total_prize'})

award_by_country_and_decade = df.groupby(['decade', 'birth_country'])['prize'].count().reset_index()

merged_df = award_by_country_and_decade.merge(total_award_by_decade, on='decade')
merged_df['ratio'] = merged_df['prize'] / merged_df['total_prize']

us_df = merged_df[merged_df['birth_country']=="United States of America"].sort_values(by='ratio', ascending=False)
max_decade_usa = us_df.head(1).decade.values[0]
max_decade_usa

np.int64(2000)

In [5]:
merged_df

,decade,birth_country,prize,total_prize,ratio
0,1900,Austrian Empire (Czech Republic),1,57,0.017544
1,1900,Austrian Empire (Italy),1,57,0.017544
2,1900,Bavaria (Germany),1,57,0.017544
3,1900,Belgium,1,57,0.017544
4,1900,British India (India),1,57,0.017544
...,...,...,...,...,...
358,2020,Russia,1,50,0.020000
359,2020,Sweden,1,50,0.020000
360,2020,USSR (now Russia),2,50,0.040000
361,2020,United Kingdom,3,50,0.060000


In [6]:
total_award_by_decade_category = df.groupby(['decade', 'category'])['prize'].count().reset_index()
total_award_by_decade_category = total_award_by_decade_category.rename(columns={'prize': 'total_prize'})

award_by_category_and_decade = df.groupby(['decade', 'category', 'sex'])['prize'].count().reset_index()
award_by_category_and_decade

merged_df2 = award_by_category_and_decade.merge(total_award_by_decade_category, on=['decade', 'category'])
merged_df2['ratio'] = merged_df2['prize'] / merged_df2['total_prize']

merged_df2 = merged_df2[merged_df2['sex']=='Female'].sort_values(by='ratio', ascending = False)

max_female_dict = {merged_df2.decade.values[0]:merged_df2.category.values[0] }

max_female_dict

{np.int64(2020): 'Literature'}

In [7]:
female_winners = df[df['sex']=='Female'].sort_values(by ='year').head(1)
first_woman_name = female_winners['full_name'].values[0]
first_woman_category = female_winners['category'].values[0]
first_woman_name, first_woman_category

('Marie Curie, née Sklodowska', 'Physics')

In [8]:
repeat_list =df[df.duplicated(subset=['full_name'])]['full_name'].unique()
repeat_list = list(repeat_list)